# **NLP3_Run3**

In [1]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.1 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [2]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [4]:
problem = """ A buyer needs to acquire 239,600,480 units of a product and is considering bids from five suppliers, labeled A through E, each of whom can only supply a portion of the total required amount.
Each vendor has proposed different pricing structures, incorporating both setup fees and variable unit costs that change based on the quantity ordered.

The buyer's objective is to allocate the order among these suppliers to minimize overall costs, accounting for both setup and unit costs.

Vendor A offers a set up cost of $3855.34 and a unit cost of $61.150 per thousand of units.
Vendor A can supply up to 33 million units.

Vendor B offers a set up cost of $125,804.84 if purchasing between 22,000,000-70,000,000 units from vendor B with a unit cost of $68.099 per thousand units.
If purchasing between 70,000,001-100,000,000 units from vendor B, the set up cost increases to $269304.84 and the unit cost sinks to $66.049 per thousand units.
If purchasing between 100,000,001-150,000,000 units from vendor B, the unit cost per thousand units further decreases to $64.099, but the set up cost increases to $464304.84.
If purchasing between 150,000,001 and 160,000,000 units from vendor B, the unit cost is $62.119 per thousand units and the set up cost equals $761304.84.

Vendor C offers set up costs of $13,456.00 and a unit cost of $62.019 per thousand units.
Vendor C can supply up to 165.6 million units. Vendor D offers set up costs of $6,583.98 and a unit cost of $72.488 for a set of thousand units.

Vendor D can supply up to 12 million units at a price of $72.488 per thousand units and with a set up cost of $6583.98.

Vendor E offers free set up if purchasing between 0 and 42 million units of vendor E with a unit price of $70.150 per thousand units.
If purchasing between 42,000,001 and 77 million units from vendor E, the unit cost starts at $68.150 per thousand units, but with every one million units purchased the price decreases at a rate of 0.05 percent. An additional set up cost of $84000 will be charged as well.

Note that zero units may be purchased from vendor B: otherwise no positive number of units less than 22,000,000 may be purchased. """

In [5]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 3
)


In [6]:
print(response1.choices[0].message.content)

Decision Variables:
Let Ai denote number of units purchased from vendor A.
Let Bj denote number of units purchased from vendor B at the jth price level (j = 1, 2, 3, 4).
Let Ck denote number of units purchased from vendor C.
Let Dl denote number of units purchased from vendor D.
Let Em denote number of units purchased from vendor E at the mth price level (m = 1, 2).

Objective Function:
Minimize
Z = 3855.34 + 61.150(Ai/1000) + 125804.84 + 68.099(B1/1000) + 269304.84 + 66.049(B2/1000) + 464304.84 + 64.099(B3/1000) + 761304.84 + 62.119(B4/1000) + 13456 + 62.019(Ck/1000) + 6583.98 + 72.488(Dl/1000) + 0 + 70.150(Em1/1000) + 84000 + (70.150 - 0.05*(Em2/1000))(Em2/1000)

Subject to the following constraints:

Ai <= 33,000,000
B1 <= 70,000,000 - 22,000,000
B2 <= 100,000,000 - 70,000,001
B3 <= 150,000,000 - 100,000,001
B4 <= 160,000,000 - 150,000,001
Ck <= 165,600,000
Dl <= 12,000,000
Em1 <= 42,000,000
Em2 <= 77,000,000 - 42,000,001

Ai + Bj + Ck + Dl + Em1 + Em2 = 239,600,480

Ai, Bj, Ck, Dl,

### **Generate Pyomo Code**

In [7]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 3
)

In [8]:
Markdown(response2.choices[0].message.content)

Here is the python code using 'pyomo' package to solve the problem:

```python
from pyomo.environ import *

model = ConcreteModel()

# Indices for the prices
prices_b = [1, 2, 3, 4]
prices_e = [1, 2]

model.A = Var(within=NonNegativeReals)
model.B = Var(prices_b, within=NonNegativeReals)
model.C = Var(within=NonNegativeReals)
model.D = Var(within=NonNegativeReals)
model.E = Var(prices_e, within=NonNegativeReals)

# Objective Function
model.cost = Objective(
    expr = 3855.34 + 61.150*(model.A/1000) 
    + 125804.84 + sum(68.099*(model.B[j])/1000 + 269304.84 + 66.049*(model.B[j])/1000 for j in prices_b)
    + 13456 + 62.019*(model.C/1000) + 6583.98 + 72.488*(model.D/1000)
    + 70.150*(model.E[1])/1000 + 84000 + 70.150*(model.E[2] - 0.05*(model.E[2]/1000)),
    sense = minimize)

# Constraints
model.con_A = Constraint(expr= model.A <= 33000000)
model.con_B = ConstraintList()
for j in prices_b:
    model.con_B.add(expr= model.B[j] <= 70000000 - (22000000*j))
model.con_C = Constraint(expr= model.C <= 165600000)
model.con_D = Constraint(expr= model.D <= 12000000)
model.con_E = ConstraintList()
for m in prices_e:
    model.con_E.add(expr= model.E[m] <= 42000000*m)
model.total = Constraint(expr = model.A + sum(model.B[j] for j in prices_b) + model.C + model.D + sum(model.E[m] for m in prices_e) == 239600480)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)
results.write()

# print the value of the variables at the optimum
print("Values of variables at optimum:")
print("A: ", model.A())
for j in prices_b:
    print("B[",j,"]: ", model.B[j]())
print("C: ", model.C())
print("D: ", model.D())
for m in prices_e:
    print("E[",m,"]: ", model.E[m]())
```

Above, you would need to install a Linear Programming (LP) Solver such as 'glpk' or 'cplex'. For 'glpk', it can be installed via the command line by the command 'apt-get install -y glpk-utils'.

### **Run the code Generated by GPT4**

In [9]:
# Install Python API for AMPL
!pip install amplpy --upgrade

from amplpy import AMPL,ampl_notebook

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 14.8 MB/s eta 0:00:00


In [10]:

_ = load_dotenv(find_dotenv('second_knitro_license.env'))
my_knitro_license = os.environ['knitro_license']

ampl = ampl_notebook(
    modules=["knitro"],  # modules to install
    license_uuid=my_knitro_license,  # license to use
)

Licensed to AMPL Community Edition License for <cevikmelis14@gmail.com>.


In [12]:
from pyomo.environ import *

model = ConcreteModel()

# Indices for the prices
prices_b = [1, 2, 3, 4]
prices_e = [1, 2]

model.A = Var(within=NonNegativeReals)
model.B = Var(prices_b, within=NonNegativeReals)
model.C = Var(within=NonNegativeReals)
model.D = Var(within=NonNegativeReals)
model.E = Var(prices_e, within=NonNegativeReals)

# Objective Function
model.cost = Objective(
    expr = 3855.34 + 61.150*(model.A/1000)
    + 125804.84 + sum(68.099*(model.B[j])/1000 + 269304.84 + 66.049*(model.B[j])/1000 for j in prices_b)
    + 13456 + 62.019*(model.C/1000) + 6583.98 + 72.488*(model.D/1000)
    + 70.150*(model.E[1])/1000 + 84000 + 70.150*(model.E[2] - 0.05*(model.E[2]/1000)),
    sense = minimize)

# Constraints
model.con_A = Constraint(expr= model.A <= 33000000)
model.con_B = ConstraintList()
for j in prices_b:
    model.con_B.add(expr= model.B[j] <= 70000000 - (22000000*j))
model.con_C = Constraint(expr= model.C <= 165600000)
model.con_D = Constraint(expr= model.D <= 12000000)
model.con_E = ConstraintList()
for m in prices_e:
    model.con_E.add(expr= model.E[m] <= 42000000*m)
model.total = Constraint(expr = model.A + sum(model.B[j] for j in prices_b) + model.C + model.D + sum(model.E[m] for m in prices_e) == 239600480)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)
results.write()

# print the value of the variables at the optimum
print("Values of variables at optimum:")
print("A: ", model.A())
for j in prices_b:
    print("B[",j,"]: ", model.B[j]())
print("C: ", model.C())
print("D: ", model.D())
for m in prices_e:
    print("E[",m,"]: ", model.E[m]())

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 10
  Number of nonzeros: 18
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: infeasible
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.018462181091308594
Values of variables at optimum:
A:  None
B[ 1 ]:  None
B[ 2 ]:  None
B[ 3 ]:  None
B[ 4 ]:  None
C:  None
D:  None
E[ 1 ]

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [14]:
from pyomo.environ import *

model = ConcreteModel()

# Indices for the prices
prices_b = [1, 2, 3, 4]
prices_e = [1, 2]

model.A = Var(within=NonNegativeReals)
model.B = Var(prices_b, within=NonNegativeReals)
model.C = Var(within=NonNegativeReals)
model.D = Var(within=NonNegativeReals)
model.E = Var(prices_e, within=NonNegativeReals)

# Objective Function
model.cost = Objective(
    expr = 3855.34 + 61.150*(model.A/1000)
    + 125804.84 + sum(68.099*(model.B[j])/1000 + 269304.84 + 66.049*(model.B[j])/1000 for j in prices_b)
    + 13456 + 62.019*(model.C/1000) + 6583.98 + 72.488*(model.D/1000)
    + 70.150*(model.E[1])/1000 + 84000 + 70.150*(model.E[2] - 0.05*(model.E[2]/1000)),
    sense = minimize)

# Constraints
model.con_A = Constraint(expr= model.A <= 33000000)
model.con_B = ConstraintList()
for j in prices_b:
    model.con_B.add(expr= model.B[1] <= 70000000 - 22000000)
    model.con_B.add(expr= model.B[2] <= 100000000 - 70000001)
    model.con_B.add(expr= model.B[3] <= 150000000 - 100000001)
    model.con_B.add(expr= model.B[4] <= 160000000 - 150000001)
model.con_C = Constraint(expr= model.C <= 165600000)
model.con_D = Constraint(expr= model.D <= 12000000)
model.con_E = ConstraintList()
for m in prices_e:
    model.con_E.add(expr= model.E[1] <= 42000000)
    model.con_E.add(expr= model.E[2] <= 77000000 - 42000001)


model.total = Constraint(expr = model.A + sum(model.B[j] for j in prices_b) + model.C + model.D + sum(model.E[m] for m in prices_e) == 239600480)

# Solve the model
solver = SolverFactory('glpk')
results = solver.solve(model)
results.write()

# print the value of the variables at the optimum
print("Values of variables at optimum:")
print("A: ", model.A())
for j in prices_b:
    print("B[",j,"]: ", model.B[j]())
print("C: ", model.C())
print("D: ", model.D())
for m in prices_e:
    print("E[",m,"]: ", model.E[m]())


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 16475399.592
  Upper bound: 16475399.592
  Number of objectives: 1
  Number of constraints: 24
  Number of variables: 10
  Number of nonzeros: 32
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.004448652267456055
# ----------------------------------------------------------
#   Solution Information
# -------------------